In [1]:
# Load libraries
# !pip install -r /work/NLP_IMDb_Exam/requirements.txt
import numpy as np
import pandas as pd
import torch
import datasets
import evaluate
import seaborn as sns
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sentence_transformers import SentenceTransformer

/home/ucloud/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-22 18:01:10.439939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734886870.454199    3989 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734886870.458519    3989 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-22 18:01:10.474940: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operatio

In [ ]:
# Read embeddings from CSV
data_path = '../Data/IMDB_Embeddings_mpnet_Base.csv'
emb_full=pd.read_csv(data_path)
emb_full = emb_full.drop(columns=['Unnamed: 0']) # Drop index collumn
emb_full

,0,1,2,3,4,5,6,7,8,9,...,761,762,763,764,765,766,767,rating,average_rating,review
0,-0.021378,0.015321,-0.013575,-0.005389,-0.059757,0.051283,-0.074764,-0.014502,-0.016757,0.028988,...,0.013569,0.029095,0.010885,0.024439,0.040906,0.006650,-0.017422,7,7.2,This movie is full of suspense. It makes you g...
1,-0.014130,0.062323,0.026934,-0.024692,-0.077509,-0.018387,-0.025898,0.018807,-0.021482,-0.008314,...,-0.004709,0.035971,-0.020709,-0.011730,0.017179,-0.015229,0.001563,5,7.2,I'll leave this review fairly concise. <br/><b...
2,-0.003857,0.008422,0.016742,-0.024959,-0.029163,0.026173,-0.080008,-0.009745,-0.013738,0.004642,...,-0.001501,0.051032,-0.039901,0.026927,0.025774,-0.022307,-0.003180,5,7.2,I give the 5/10 out of the credit I owe to the...
3,0.045045,0.027935,0.001183,0.004430,-0.021354,-0.008525,-0.069501,-0.007163,-0.031054,-0.004886,...,0.005012,0.026980,-0.038002,0.004601,0.065031,0.030194,-0.015002,5,7.2,"First of all, I must say that I was expecting ..."
4,0.004422,0.020985,-0.008421,-0.008166,-0.033740,0.053173,-0.058947,-0.045884,-0.018905,0.013832,...,0.012538,0.006282,0.015431,0.013172,0.072710,0.022397,0.021586,7,7.2,I've always loved movies with strong atmospher...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.017325,0.015409,0.011389,-0.030874,-0.018250,0.020404,-0.055944,0.007315,-0.035929,-0.001511,...,-0.016607,-0.014120,-0.112227,0.029976,0.030388,0.066537,-0.032054,8,7.7,SPOILER Being John Malkovich is a thought-prov...
99996,0.009220,0.019475,-0.006678,-0.018930,-0.008402,0.002885,-0.064432,0.031742,-0.042335,-0.008312,...,0.018085,0.008839,-0.077946,-0.008324,0.037355,0.056813,-0.020391,8,7.7,i didn't know what to expect from this movie. ...
99997,0.028693,0.061061,-0.006029,-0.011065,-0.043209,0.006381,-0.017647,0.000348,-0.025219,-0.017800,...,-0.011471,0.018212,-0.036587,-0.003243,0.047629,0.086389,-0.029291,5,7.7,This is the first time in a good while that I'...
99998,-0.024934,0.057979,-0.007299,-0.010783,-0.001762,0.006831,-0.065981,0.017861,-0.032820,0.010486,...,0.020986,-0.001424,-0.079580,0.032276,0.032845,0.067550,-0.004554,10,7.7,I admit that I wanted to see this film for the...


In [12]:
from sklearn.model_selection import train_test_split
# Remove rest of unecessary columns before splitting:
split = emb_full.drop(columns=['review'])
split = split.drop(columns=['rating'])
split = train_test_split(split, train_size=0.7)
finaleval=split[1]
subset=split[0]

train_features = subset.drop(columns=["average_rating"]).to_numpy()
train_labels = np.float32(subset['average_rating'].to_numpy())
eval_features = finaleval.drop(columns=["average_rating"]).to_numpy()
eval_labels = np.float32(finaleval[['average_rating']].to_numpy())

# Create a linear regression of entire embedding space:

In [13]:
# !pip install statsmodels
import statsmodels.api as sm
#Regression Model
X = sm.add_constant(train_features)# adding a constant
model=sm.OLS(train_labels, X).fit()

In [14]:
# Predictive accuracy:
X_test = sm.add_constant(eval_features) # add intercept to test_data
predictions = model.predict(X_test) # predict

In [15]:
# Summary Statistics
print("R-squared:", model.rsquared)
eval_labels = eval_labels.ravel()
results_df = pd.DataFrame({
    'True Labels': eval_labels,
    'Predicted Labels': predictions
})
print("MSE:", np.mean((results_df['True Labels'] - results_df['Predicted Labels'])**2))
print("Corr:", results_df.corr().iloc[0,1])
results_df

R-squared: 0.6206553365202413
MSE: 0.22477601261939023
Corr: 0.7831740836537188


,True Labels,Predicted Labels
0,8.3,7.325988
1,5.9,6.428761
2,7.7,7.838576
3,6.2,6.894935
4,8.5,8.878840
...,...,...
29995,8.5,7.899920
29996,8.4,7.957676
29997,8.0,7.971181
29998,8.4,7.635095


# Create an ICA model:

In [7]:
from sklearn.decomposition import FastICA
ica = FastICA(n_components=20, whiten="arbitrary-variance") # Define ICA
My_ICA_fit = ica.fit_transform(train_features) # Fit ICA to data

In [8]:
# Get estimated mixing matrix
mixMatrixEst = ica.mixing_

# Map embeddings onto ICA-space
train_ICA_features = pd.DataFrame(train_features @ mixMatrixEst)
eval_ICA_features = pd.DataFrame(eval_features @ mixMatrixEst)

In [9]:
# !pip install statsmodels
import statsmodels.api as sm
#Regression Model
X = sm.add_constant(train_features)# adding a constant
model=sm.OLS(train_labels, X).fit()

In [10]:
# Predictive accuracy:
X_test = sm.add_constant(eval_features) # add intercept to test_data
predictions = model.predict(X_test) # predict

In [11]:
# Summary Statistics
print("R-squared:", model.rsquared)
eval_labels = eval_labels.ravel()
results_df = pd.DataFrame({
    'True Labels': eval_labels,
    'Predicted Labels': predictions
})
print("MSE:", np.mean((results_df['True Labels'] - results_df['Predicted Labels'])**2))
print("Corr:", results_df.corr().iloc[0,1])
results_df

R-squared: 0.6429708010818195
MSE: 3.0834466156084974
Corr: 0.788516107348885


,True Labels,Predicted Labels
0,9.0,8.280484
1,9.0,8.867151
2,7.0,7.090550
3,9.0,8.568203
4,8.0,7.600641
...,...,...
29995,9.0,6.310360
29996,10.0,9.743711
29997,10.0,9.206493
29998,6.0,5.554232
